<a href="https://colab.research.google.com/github/mingmingbupt/tensorflow/blob/master/tf1_customized_estimator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
%tensorflow_version 1.x
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

TensorFlow is already loaded. Please restart the runtime to change versions.
1.15.2
sys.version_info(major=3, minor=6, micro=9, releaselevel='final', serial=0)
matplotlib 3.2.1
numpy 1.18.2
pandas 1.0.3
sklearn 0.22.2.post1
tensorflow 1.15.2
tensorflow.python.keras.api._v1.keras 2.2.4-tf


In [15]:
#如何在colab上使用在google cloud上面的文件
import os
from google.colab import drive
drive.mount('/content/drive')

#path = "/content/drive/My Drive/Colab Notebooks/tensorflow/Tensorflow2.0--rumendaoshijian/chapter_5_tf1"
path = "/content/drive/My Drive/Colab Notebooks/tensorflow/Tensorflow2.0--rumendaoshijian/chapter_5_tf1（带数据）"

os.chdir(path)
os.listdir(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['tf1_customized_estimator.ipynb',
 'data',
 'tf1_dense_network.ipynb',
 'tf1_dataset.ipynb',
 'tf1_initialized_dataset.ipynb',
 'customized_estimator']

In [16]:
# https://storage.googleapis.com/tf-datasets/titanic/train.csv
# https://storage.googleapis.com/tf-datasets/titanic/eval.csv
train_file = "./data/titanic/train.csv"
eval_file = "./data/titanic/eval.csv"

train_df = pd.read_csv(train_file)
eval_df = pd.read_csv(eval_file)

print(train_df.head())
print(eval_df.head())

   survived     sex   age  ...     deck  embark_town  alone
0         0    male  22.0  ...  unknown  Southampton      n
1         1  female  38.0  ...        C    Cherbourg      n
2         1  female  26.0  ...  unknown  Southampton      y
3         1  female  35.0  ...        C  Southampton      n
4         0    male  28.0  ...  unknown   Queenstown      y

[5 rows x 10 columns]
   survived     sex   age  ...     deck  embark_town  alone
0         0    male  35.0  ...  unknown  Southampton      y
1         0    male  54.0  ...        E  Southampton      y
2         1  female  58.0  ...        C  Southampton      y
3         1  female  55.0  ...  unknown  Southampton      y
4         1    male  34.0  ...        D  Southampton      y

[5 rows x 10 columns]


In [17]:
y_train = train_df.pop('survived')
y_eval = eval_df.pop('survived')

print(train_df.head())
print(eval_df.head())
print(y_train.head())
print(y_eval.head())

      sex   age  n_siblings_spouses  parch  ...  class     deck  embark_town alone
0    male  22.0                   1      0  ...  Third  unknown  Southampton     n
1  female  38.0                   1      0  ...  First        C    Cherbourg     n
2  female  26.0                   0      0  ...  Third  unknown  Southampton     y
3  female  35.0                   1      0  ...  First        C  Southampton     n
4    male  28.0                   0      0  ...  Third  unknown   Queenstown     y

[5 rows x 9 columns]
      sex   age  n_siblings_spouses  parch  ...   class     deck  embark_town alone
0    male  35.0                   0      0  ...   Third  unknown  Southampton     y
1    male  54.0                   0      0  ...   First        E  Southampton     y
2  female  58.0                   0      0  ...   First        C  Southampton     y
3  female  55.0                   0      0  ...  Second  unknown  Southampton     y
4    male  34.0                   0      0  ...  Second     

In [18]:
train_df.describe()

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [19]:
categorical_columns = ['sex', 'n_siblings_spouses', 'parch', 'class',
                       'deck', 'embark_town', 'alone']
numeric_columns = ['age', 'fare']

feature_columns = []
for categorical_column in categorical_columns:
    vocab = train_df[categorical_column].unique()
    print(categorical_column, vocab)
    feature_columns.append(
        tf.feature_column.indicator_column(
            tf.feature_column.categorical_column_with_vocabulary_list(
                categorical_column, vocab)))

for categorical_column in numeric_columns:
    feature_columns.append(
        tf.feature_column.numeric_column(
            categorical_column, dtype=tf.float32))

sex ['male' 'female']
n_siblings_spouses [1 0 3 4 2 5 8]
parch [0 1 2 5 3 4]
class ['Third' 'First' 'Second']
deck ['unknown' 'C' 'G' 'A' 'B' 'D' 'F' 'E']
embark_town ['Southampton' 'Cherbourg' 'Queenstown' 'unknown']
alone ['n' 'y']


In [0]:
def make_dataset(data_df, label_df, epochs = 10, shuffle = True,
                 batch_size = 32):
    dataset = tf.data.Dataset.from_tensor_slices(
        (dict(data_df), label_df))
    if shuffle:
        dataset = dataset.shuffle(10000)
    dataset = dataset.repeat(epochs).batch(batch_size)
    return dataset.make_one_shot_iterator().get_next() 
    #在tensorflow1.0里面，dataset需要调用iterator,然后调用get_next，来获取具体值的tensor，返回的值就可以在自定义的model_fn里直接使用了

In [21]:
output_dir = "customized_estimator"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

def model_fn(features, labels, mode, params): #定义函数，四个参数
    # model 就是运行时状态runtime state: Train, Eval, Predict（线上服务预测的时候）
    # train和eval的时候是可以得到准确率和loss，但是在predict的时候是得不到的
    # 在model不同的时候，得到不同的返回值
    input_for_next_layer = tf.feature_column.input_layer( # tf.feature_column.input_layer他和keras.layers.denseFeatures是一样的作用
    #他会把我们的features给解析出来，利用的就是params['feature_columns']参数
        features, params['feature_columns']) # 第一层，把features解析出来
    for n_unit in params['hidden_units']: #隐含层，使用循环来构建全连接层
        input_for_next_layer = tf.layers.dense(input_for_next_layer,
                                               units = n_unit,
                                               activation = tf.nn.relu)
    logits = tf.layers.dense(input_for_next_layer,
                             params['n_classes'],
                             activation = None) #无激活函数
    predicted_classes = tf.argmax(logits, 1)  #获得我们预测出来的类
    
    if mode == tf.estimator.ModeKeys.PREDICT: #predict
       #给predict mode返回结果，predictions是一个字典
        predictions = {
            "class_ids": predicted_classes[:, tf.newaxis], 
            #要求返回的是一个 (n * 1)的一个矩阵，这里predicted_classes是一个长度为n的向量，所以我要给他扩展一个维度
            "probabilities": tf.nn.softmax(logits), #也就是他输出的概率分布，可以通过给logits做softmax获得
            "logits": logits
        }#这样就构建好了我们的字典
        #对于自定义的estimator来说，他的自定义的model_fn都必须返回的是一个tf.estimator.EstimatorSpec的对象
        #这个estimator就定义了session run要run那些op,这里就是在predictions里面的参数，class_ids，probabilities，logits
        return tf.estimator.EstimatorSpec(mode, predictions = predictions)
    
    #计算我们的损失函数，使用sparse_softmax_cross_entropy来进行计算
    loss = tf.losses.sparse_softmax_cross_entropy(labels = labels, logits = logits)
    #计算准确率。之前的例子里我们使用了比较原始的方法来计算准确率。就是想让predicted_classes和labels之间对比，在对比的结果上呢，
    #我们去取平均，得到准确率。在这里我们使用一个高层的封装，tf.metrics.accuracy，这个封装的好处就是他可以帮我们计算累计的准确率
    #在之前的例子中eval的时候，我们是在不同batch上分别求准确率，然后再去求平均。然而在 tf.metrics.accuracy这里呢，就不需要了
    #因为我的这个tf.metrics.accuracy可以帮我去做累计
    accuracy = tf.metrics.accuracy(labels = labels, predictions = predicted_classes, name = "acc_op")
    metrics = {"accuracy": accuracy} #定义了我们的指标
    if mode == tf.estimator.ModeKeys.EVAL: #eval
        return tf.estimator.EstimatorSpec(mode, loss = loss,eval_metric_ops = metrics)
    #eval mode下返回的依然是tf.estimator.EstimatorSpec，有三个参数，mode,loss,eval_metric_ops
    #train mode                                   
    optimizer = tf.train.AdamOptimizer()
    train_op = optimizer.minimize(loss, global_step = tf.train.get_global_step())
    #train mode下需要参数mode loss train_op
    return tf.estimator.EstimatorSpec(mode, loss = loss, train_op = train_op)

#用定义好的函数去初始化我们的estimator，用自定义的函数去初始化estimator也非常简单
#需要调用tf.estimator.Estimator， 第一个是model_fn,就是我们上面定义的model_fn
#model_dir
#params是个字典，通过model_fn(features, labels, mode, params)里面的params传到model_fn当中来
#这样就可以构建一个自定义的estimator了
estimator = tf.estimator.Estimator(
    model_fn = model_fn,
    model_dir = output_dir,
    params = {
        "feature_columns": feature_columns, #之前的feature_columns传进去
        "hidden_units": [100, 100],#神经网络的层次，单元数
        "n_classes": 2 #类别数2类
    })

#model_fn完成了 和 estimator初始化好了
estimator.train(input_fn = lambda : make_dataset(
    train_df, y_train, epochs = 100))#参数只有input_fn

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'customized_estimator', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8d30683f28>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Use `for ... in dataset:`

In [22]:
estimator.evaluate(lambda : make_dataset(
    eval_df, y_eval, epochs = 1))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-04-06T07:02:11Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from customized_estimator/model.ckpt-1960
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2020-04-06-07:02:11
INFO:tensorflow:Saving dict for global step 1960: accuracy = 0.7916667, global_step = 1960, loss = 0.4775967
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1960: customized_estimator/model.ckpt-1960


{'accuracy': 0.7916667, 'global_step': 1960, 'loss': 0.4775967}